### Summery
<pre>
Author           : Anjana Tiha
Project Name     : Histopathologic Cancer Detection using Convolutional Neural Network, and Transfer Learning.
Description      : 1. Detected Cancer from Histopathologic images by retraining pretrained model “InceptionV3” with                            250000+ images of X-ray (6GB).
                   2. For retraining, removed output layers, freezed first few layers and Fine-tuned model for two new label                   classes (Cancer and Normal).
                   3. Attained testing accuracy 69.55 and loss 1.10.
Method           : 
Tools/Library    : Python, Keras, PyTorch, TensorFlow
Version History  : 1.0.0.0
Current Version  : 1.0.0.0
Last Update      : 11.28.2018
Comments         : Please use Anaconda editor for convenience.
</pre>

#### Code
<pre>
GitHub Link      : <a href=https://github.com/anjanatiha/Histopathologic-Cancer-Detection>Histopathologic Cancer Detection(GitHub)</a>
GitLab Link      : <a href=https://gitlab.com/anjanatiha/Histopathologic-Cancer-Detection>Histopathologic Cancer Detection(GitLab)</a>
Portfolio        : <a href=https://anjanatiha.wixsite.com/website>Anjana Tiha's Portfolio</a>
</pre>

#### Dataset
<pre>
Dataset Name     : Histopathologic Cancer Detection
Dataset Link     : <a href=https://www.kaggle.com/c/histopathologic-cancer-detection>Histopathologic Cancer Detection (Kaggle)</a>
                 : <a href=https://github.com/basveeling/pcam> PatchCamelyon (PCam) (GitHub)</a>
                 : <a href=https://camelyon16.grand-challenge.org/Data>CAMELYON16 challenge Dataset (Original Dataset)</a>
                 
Original Paper   : <a href=https://jamanetwork.com/journals/jama/fullarticle/2665774>Diagnostic Assessment of Deep Learning Algorithms for Detection of Lymph Node Metastases in Women With Breast Cancer</a> 
                   Authors: Babak Ehteshami Bejnordi, Mitko Veta, Paul Johannes van Diest 
                   JAMA (The Journal of the American Medical Association)
                   <cite>
                   Ehteshami Bejnordi B, Veta M, Johannes van Diest P, et al. Diagnostic Assessment of Deep Learning                        Algorithms for Detection of Lymph Node Metastases in Women With Breast Cancer. JAMA.                                     2017;318(22):2199–2210. doi:10.1001/jama.2017.14585
                   </cite>
</pre>


#### Library/Tools Version
- Python - v3.6.7
- argparse
- random
- numpy
- shutil
- gc
- re
- Keras - 2.2.4
- Keras-preprocessing - v1.0.5
- TensorFlow - 1.12
- PIL/Pillow - 5.1.0
- Matplotlib - 2.2.2
- scikit-learn - 0.19.1
- mlxtend - 0.14.0

#### Commands / Running Instruction
<pre>
tensorboard --logdir=logs
%config IPCompleter.greedy=True
</pre>

<pre>
<b>Dataset Details</b>
Dataset Name            : Histopathologic Cancer Detection
Number of Class         : 2
Number/Size of Images   : Total      : 220,025 (5.72 Gigabyte (GB))
                          Training   : 132,016 (3.43 Gigabyte (GB))
                          Validation : 44,005  (1.14 Gigabyte (GB))
                          Testing    : 44,004  (1.14 Gigabyte (GB))

<b>Model Parameters</b>
Machine Learning Library: Keras
Base Model              : InceptionV3
Optimizers              : Adam
Loss Function           : categorical_crossentropy

<b>Training Parameters</b>
Batch Size              : 32
Number of Epochs        : 20
Training Time           : 1 day and 8 hour (33 Hours)

<b>Output (Prediction/ Recognition / Classification Metrics)</b>
<!--<b>Validation</b>-->
<b>Testing</b>
Accuracy                : 69.55%
Loss                    : 1.10
<!--Precision               : -->
Recall                  : 
<!--Specificity             : -->
</pre>

## Import Libraries

In [ ]:
from __future__ import print_function

import argparse
import sys
import os

import random

import time
import datetime

from collections import Counter
import numpy as np

import shutil
import inspect

import gc

import re

import keras
from keras import models
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, GlobalAveragePooling1D
from keras import optimizers

from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau

from keras import backend as K

import tensorflow as tf

from IPython.display import display
from PIL import Image

import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from mlxtend.plotting import plot_confusion_matrix



## Functions

### File Function

In [ ]:
# Creates directory, if directory exists removes if remove parameter is set to True 
def create_directory(directory_path, remove=False):
    if remove and os.path.exists(directory_path):
        try:
            shutil.rmtree(directory_path)
            os.mkdir(directory_path)
        except:
            print("Could not remove directory : ", directory_path)
            return False
    else:
        try:
            os.mkdir(directory_path)
        except:
            print("Could not create directory: ", directory_path)
            return False
        
    return True

# Removes directory, if directory exists 
def remove_directory(directory_path):
    if os.path.exists(directory_path):
        try:
            shutil.rmtree(directory_path)
        except:
            print("Could not remove directory : ", directory_path)
            return False
        
    return True

# Deletes file, if file exists 
def remove_file(filename):
    if os.path.exists(filename):
        try:
            os.remove(filename)
        except:
            print("Could not remove file : ", filename)
            return False
        
    return True



### Time Function

In [ ]:
# print date and time for given type of representation
def date_time(x):
    if x==1:
        print('Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))
    if x==2:    
        print('Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now()))
    if x==3:  
        print('Date now: %s' % datetime.datetime.now())
    if x==4:  
        print('Date today: %s' % datetime.date.today())  

### Debug Functions

In [ ]:
# prints a integer for degugging
def debug(x):
    print("-"*40, x, "-"*40)  

### String Function

In [ ]:
# Removes everything except alphabetical and selected characters from name string
def name_correct(name):
    return re.sub(r'[^a-zA-Z,:]', ' ', name).title()

### Data Visualization Function

In [ ]:
# count number of files in each subdirectory of a directory
def subdirectory_file_count(master_directory):
    subdirectories = os.listdir(master_directory)
    subdirectory_count = len(subdirectories)

    subdirectory_names = []
    subdirectory_file_counts = []

    for subdirectory in subdirectories:
        current_directory = os.path.join(master_directory, subdirectory)
        file_count = len(os.listdir(current_directory))
        subdirectory_names.append(subdirectory)
        subdirectory_file_counts.append(file_count)
    
    return subdirectory_names, subdirectory_file_counts
               

# show barplot
def bar_plot(x, y, title, xlabel, ylabel, figsize=(10,8), title_fontsize = 14, label_fontsize=12, subplot_no=0):
    if subplot_no:
        plt.subplot(subplot_no)
    sns.barplot(x=x, y=y)
    plt.title(title, fontsize=title_fontsize)
    plt.xlabel(xlabel, fontsize=label_fontsize)
    plt.ylabel(ylabel, fontsize=label_fontsize)
    plt.xticks(range(len(x)), x)
    

# show bar plot for count of labels in subdirectory of a directory
def count_bar_plot(master_directory, title, xlabel, ylabel, figsize=(10,8), title_fontsize = 14, label_fontsize=12, subplot_no=0):
    dir_name, dir_file_count = subdirectory_file_count(master_directory)
    x=dir_name
    y=dir_file_count
    bar_plot(x, y, title, xlabel, ylabel, figsize=fig_size, title_fontsize=title_fontsize, label_fontsize=label_fontsize, subplot_no=subplot_no)
    
    
# show bar plot for count of labels in subdirectory of a training, validation, testing directory    
def show_train_val_test(training_dir, validation_dir, testing_dir, title, xlabel, ylabel, figsize=(10,8), title_fontsize = 14, label_fontsize=12):
    plt.figure(figsize=fig_size)
    count_bar_plot(training_dir, title +" (Training)", xlabel, ylabel, fig_size, title_fontsize, label_fontsize, subplot_no=131)
    count_bar_plot(validation_dir, title +" (Validation)", xlabel, ylabel, fig_size, title_fontsize, label_fontsize, subplot_no=132)
    count_bar_plot(testing_dir, title +" (Testing)", xlabel, ylabel, fig_size, title_fontsize, label_fontsize, subplot_no=133)
    plt.show()

### Image Preprocessing Function

In [ ]:
# Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches)
def get_transformed_image_batch(directory, target_size, classes, class_mode='categorical', batch_size=1, shuffle=True, rescale=None, shear_range=0.0, zoom_range=0.0, horizontal_flip=False, validation_split=0.0):       
    datagen = ImageDataGenerator(
            rescale=rescale,
            shear_range=shear_range,
            zoom_range=zoom_range,
            horizontal_flip=horizontal_flip,
            validation_split=validation_split)     
    
    image_generator = datagen.flow_from_directory(
            directory,
            target_size=target_size,
            classes = classes,
            class_mode=class_mode,
            batch_size=batch_size)
    return image_generator

### Data Label Preprocessing Function

In [ ]:
# adjust class weights for imbalanced dataset of classes of images
def get_class_weight(y):
    counter = Counter(y)                          
    max_val = float(max(counter.values()))     
    class_weight = {class_id : max_val/num_images for class_id, num_images in counter.items()}   
    return class_weight

### Tensorflow Graph Function

In [ ]:
# reset tensorflow graph tp free up memory and resource allocation 
def reset_graph(model=None):
    try:
        del model
    except:
        return False
    tf.reset_default_graph()
    K.clear_session()
    gc.collect()
    return True


# reset callbacks 
def reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint=None
    reduce_lr = None
    early_stopping = None
    tensorboard = None

In [ ]:
def swish_activation(x):
    return (K.sigmoid(x) * x)

def basic_model(optimizer, loss, metrics, input_shape=(3,150,150), activation='relu', activation2='sigmoid', padding="same", padding2="valid", pool_size=(2, 2), dilation_rate=(2, 2)):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation=activation, padding=padding, input_shape=input_shape))
    model.add(Conv2D(16, (3, 3), padding=padding, activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(32, (3, 3), activation=activation, padding=padding, input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), padding=padding, activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(64, (3, 3), activation=activation, padding=padding))
    model.add(Conv2D(64, (3, 3), padding=padding, activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(96, (3, 3), dilation_rate=dilation_rate, activation=activation, padding=padding))
    model.add(Conv2D(96, (3, 3), padding2=padding2, activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Conv2D(128, (3, 3), dilation_rate=dilation_rate, activation=activation, padding=padding))
    model.add(Conv2D(128, (3, 3), padding2=padding2, activation=activation))
    model.add(MaxPooling2D(pool_size=pool_size))

    model.add(Flatten())

    model.add(Dense(64, activation=swish_activation))
    model.add(Dropout(0.4))
    model.add(Dense(2 , activation=activation2))

    model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)

    print(model.summary())

In [ ]:
def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(2, activation='softmax', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model

### Model Initialization Function
#### Load and Configure Model Function InceptionV3 for Fine-Tuning with New Class Labels
<p>1. Imports Pretrained model InceptionV3 <br>
   2. Disabled training on first few layers <br>
   3. Enabled training on top and output layers<br>
   4. Adjust output Dense Layer to number of Image Classes <br>
</p>

In [ ]:
# Load and configure model InceptionV3 for fine-tuning with new class labels
def get_inception_model(train_generator, validation_generator, epochs, verbose, optimizer, loss, metrics, tensorboard, callbacks, num_class, include_top=False, non_trainable_index=249, print_layers = False):    
    # create the base pre-trained model
    base_model = InceptionV3(weights='imagenet', include_top=include_top)

    # add a global spatial average pooling layer
    # Setting model layers specially output layer with class number
    x = base_model.output
    
#     x = Dropout(0.5)(x)
    
    x = GlobalAveragePooling2D()(x)
    
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    
    # and a logistic layer -- let's say we have 2 classes

    # softmax for multi-class
    predictions = Dense(num_class, activation='softmax')(x) 
    
    # sigmoid for 2 class or binary class
    # predictions = Dense(num_class, activation='sigmoid')(x) 

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
    
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False
     


    
    if callbacks:
        # compile model with loss, optimizer and metrics 
        model.compile(optimizer, loss=loss, metrics=metrics)

        tensorboard.set_model(model) 
    
        # train the model on the new data for a few epochs
        model.fit_generator(train_generator,
                            steps_per_epoch = len(train_generator),
                            epochs=epochs,
                            # verbose=verbose, 
                            callbacks=callbacks,
                            validation_data=validation_generator,
                            validation_steps=len(validation_generator),
                            class_weight = class_weight)

    # at this point, the top layers are well trained and we can start fine-tuning
    # convolutional layers from inception V3. We will freeze the bottom N layers
    # and train the remaining top layers.

    # let's visualize layer names and layer indices to see how many layers
    # we should freeze:
    if print_layers:
        for i, layer in enumerate(base_model.layers):
            print(i, layer.name)

    # Freeze or set first few layers as untrainable
    # Unfreeze or set rest of the layers as trainable
    for layer in model.layers[:non_trainable_index]:
        layer.trainable = False
    for layer in model.layers[non_trainable_index:]:
        layer.trainable = True
        
    if not callbacks:
        model.compile(optimizer, loss=loss, metrics=metrics)
        tensorboard.set_model(model) 
        
    model.summary()
        
    return model


### Training Performance Visualization Function

In [ ]:
# Plot training & validation performance
def plot_history(history, plot_val, title, xlabel, ylabel, legend=[['Train', 'Val'], ['Train', 'Val']], fig_size=(10,8), title_fontsize = 14, label_fontsize=12):
    plt.figure(figsize=fig_size)
    
    plt.subplot(121)
    plt.plot(history.history[plot_val[0][0]])
    plt.plot(history.history[plot_val[0][1]])
    plt.title(title[0], fontsize=title_fontsize)
    plt.ylabel(ylabel[0], fontsize=label_fontsize)
    plt.xlabel(xlabel[0], fontsize=label_fontsize)
    plt.legend(legend[0], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history.history[plot_val[1][0]])
    plt.plot(history.history[plot_val[1][1]])
    plt.title(title[1], fontsize=title_fontsize)
    plt.ylabel(ylabel[1], fontsize=label_fontsize)
    plt.xlabel(xlabel[1], fontsize=label_fontsize)
    plt.legend(legend[1], loc='upper left')
    plt.show()

### Testing Performance Report Function

In [ ]:
def model_evaluate(model, test_generator, print_report=False):
    if len(test_generator)>1:
        result = model.evaluate_generator(generator=test_generator, steps=len(test_generator))
    else:
        result = model.evaluate_generator(generator=test_generator, steps=len(test_generator), verbose=1)
        
    
    accuracy = result[1]*100
    loss = result[0]
    
    if print_report:
        print("%s%.2f%s"% ("Accuracy: ", accuracy, "%"))
        print("%s%.2f"% ("Loss: ", loss))
    
    return accuracy, loss


def predict_report(model, test_generator, classes, print_report=False):
    if len(test_generator)>1:
        y_preds = model.predict_generator(test_generator, steps=len(test_generator))
    else:
        y_preds = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)
        
    y_classes = y_preds.argmax(axis=-1)

    CM = confusion_matrix(test_generator.classes, y_classes)
    cls_report_print = classification_report(test_generator.classes, y_classes, target_names=classes)
    cls_report = classification_report(test_generator.classes, y_classes, target_names=classes, output_dict=True)
    
    if print_report: 
        print(cls_report_print)
    return y_preds, y_classes, CM, cls_report, cls_report_print


### Performance Metrics Function

In [ ]:
# reverse classification report for reporting negetive classes (0)
def reverse_pos_neg(CM, print_bool):
    tp=CM[0][0]
    fp=CM[0][1]
    fn=CM[1][0]
    tn=CM[1][1]
    if print_bool:
        print(tp, fp, tn, fn, tn)
    return [tp, fp, tn, fn, tn]

# reverse and report classification report for reporting negetive classes (0)
def report(CM, reverse):
    if not reverse:
        tn, fp, fn, tp = CM.ravel()

    else:
        tp=CM[0][0]
        fp=CM[0][1]
        fn=CM[1][0]
        tn=CM[1][1]
    
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    
    print("Recall of the model is {:.2f}".format(recall))
    print("Precision of the model is {:.2f}".format(precision))
    
    return precision, recall

### Testing Visualization Function

In [ ]:
def line_plot_over_epochs(array, title, xlabel="Epoch", ylabel="Value", title_fontsize=14, label_fontsize=12, subplot_no=0):
    x_axis_arr = np.arange(len(array))
    if subplot_no:
        plt.subplot(subplot_no)
    plt.title(title, fontsize=title_fontsize)
    plt.plot(x_axis_arr, array)
    plt.xlabel(xlabel, fontsize=label_fontsize)
    plt.ylabel(ylabel, fontsize=label_fontsize)
    
def line_plot_over_epochs_loss_acc(array, title, fig_size=(10, 8), xlabel=["Epoch", "Epoch"], ylabel=["Value","Value"], title_fontsize=14, label_fontsize=12):
    plt.figure(figsize=fig_size)
    line_plot_over_epochs(array[0], title[0], xlabel=xlabel[0], ylabel=ylabel[0], title_fontsize=title_fontsize, label_fontsize=label_fontsize, subplot_no=121)
    line_plot_over_epochs(array[1], title[1], xlabel=xlabel[1], ylabel=ylabel[1], title_fontsize=title_fontsize, label_fontsize=label_fontsize, subplot_no=122)
    plt.show()
    
def show_confusion_matrix(test_generator, y_classes, classes, figsize=(10,8), stick_fontsize=12):
    CM = confusion_matrix(test_generator.classes, y_classes)
    fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=figsize, hide_ticks=True,cmap=plt.cm.Blues)
    plt.xticks(range(len(classes)), classes, fontsize=stick_fontsize)
    plt.yticks(range(len(classes)), classes, fontsize=stick_fontsize)
    plt.show()
    return CM

In [ ]:
def test_model(model_filename, details, report_type, classes, class_name):
    results1 = {}
    results2 = {}
        
    model_path = model_dir+"\\"+model_filename
        
    if not os.path.isdir(model_path):
        return None
    
    reset_graph(model)

    model = keras.models.load_model(model_path)

    accuracy, loss =  model_evaluate(model, test_generator, print_report=False)
    y_preds, y_classes, CM, cls_report, cls_report_print = predict_report(model, test_generator, classes)


    precision = cls_report[class_name]['precision'] *100
    recall =  cls_report[class_name]['recall'] *100
    f1_score =  cls_report['weighted avg']['f1-score'] *100



    results1[model_filename] = [accuracy, loss]
    results2[model_filename] = [CM, cls_report, cls_report_print]

    print("%s%s"%("Model File: ", model_filename))
    print("*"*80)
    show_confusion_matrix(test_generator, y_classes, classes)
    print(cls_report_print)
    print("%s%.2f%s"% ("Current Accuracy: ", accuracy, "%"))
    print("%s%.2f"% ("Current Loss: ", loss))
    print("%s%.2f%s"% ("Current Precision: ", precision, "%"))
    print("%s%.2f%s"% ("Current Recall: ", recall, "%"))
    print("%s%.2f%s"% ("Current F1_score: ", f1_score, "%"))

    print("-"*80)
    print("-"*80)

    
    print("Testing dataset evaluation and prediction report generation complete")

    report = {"Accuracy" : accuracy, 
              "Loss" : loss,
              "Precision" : precision,
              "Recall": recall,
              "F1-Score":f1_score}


    return results1, results2, report

### Testing Function

In [ ]:
def test_all_models(model_dir, details, report_type, classes, class_name):
    results1 = {}
    results2 = {}
    
    filenames=[]
    
    accuracy_list=[]
    loss_list=[]
    precision_list=[]
    recall_list=[]
    f1_score_list=[]
    
    
    best_accuracy=0
    best_accuracy_file=""
    
    best_loss=1000
    best_loss_file=""
    
    
    best_precision=0
    best_precision_file=""
    
    best_recall=0
    best_recall_file=""
    
    best_f1_score=0
    best_f1_score_file=""
    

    model_files = os.listdir(model_dir)
    
    model = None
    reset_graph(model)
    
    i=0
    
    for model_filename in model_files:
        
        model_path = model_dir+"\\"+model_filename
        
        if not os.path.isdir(model_path):
            reset_graph(model)

            model = keras.models.load_model(model_path)
            
            current_accuracy, current_loss =  model_evaluate(model, test_generator, print_report=False)
            y_preds, y_classes, CM, cls_report, cls_report_print = predict_report(model, test_generator, classes)

            
            current_precision = cls_report[class_name]['precision'] *100
            current_recall =  cls_report[class_name]['recall'] *100
            current_f1_score =  cls_report['weighted avg']['f1-score'] *100
            
            filenames.append(model_file)
            
            accuracy_list.append(current_accuracy)
            loss_list.append(current_loss)
            
            precision_list.append(current_precision)
            recall_list.append(current_recall)
            f1_score_list.append(current_f1_score)
            
            
            results1[model_filename] = [current_accuracy, current_loss]
            results2[model_filename] = [CM, cls_report, cls_report_print]

                
            if current_accuracy>=best_accuracy:
                best_accuracy=current_accuracy
                best_accuracy_file=model_filename

            if current_loss<=best_loss:
                best_loss=current_loss
                best_loss_file=model_filename
                
            
            if current_precision>=best_precision:
                best_precision=current_precision
                best_precision_file=model_filename

            if current_recall>=best_recall:
                best_recall=current_recall
                best_recall_file=model_filename
                
            if current_f1_score>=best_f1_score:
                best_f1_score=current_f1_score
                best_f1_score_file=model_filename
                    

            if details or i%5==0:
                print("%s%s"%("Model No: ", i+1))
                print("%s%s"%("Model File: ", model_filename))
                print("*"*80)
                show_confusion_matrix(test_generator, y_classes, classes)
                print(cls_report_print)
                print("%s%.2f%s"% ("Current Accuracy: ", current_accuracy, "%"))
                print("%s%.2f"% ("Current Loss: ", current_loss))
                print("%s%.2f%s"% ("Current Precision: ", current_precision, "%"))
                print("%s%.2f%s"% ("Current Recall: ", current_recall, "%"))
                print("%s%.2f%s"% ("Current F1_score: ", current_f1_score, "%"))

                print("-"*80)
                print("-"*80)

            i+=1
    print("Testing dataset evaluation and prediction report generation complete")
    
    report = {"Best Accuracy" : [best_accuracy, best_accuracy_file], 
                   "Best Loss" : [best_loss, best_loss_file],
                   "Best Precision" : [best_precision, best_precision_file],
                   "Best Recall": [best_recall, best_recall_file],
                   "Best F1-Score":[best_f1_score, best_f1_score_file]}
    
    
    return results1, results2, report

# Preprocessing

### Reset Graph

In [ ]:
x=1
date_time(x)
reset_graph(model)

### Set Path for Train, Validation and Test Dataset

In [ ]:
# Configure input/ output directory
# Configure training, validation, testing directory

input_directory = r"data/input/"
output_directory = r"data/output/"

training_dir = input_directory+ r"train_final"
testing_dir = input_directory+ r"test_final"
validation_dir = input_directory+ r"validation_final"

### Data Visualization

In [ ]:
xlabel="Count"
ylabel="CaseType"
fig_size = (18,4)
# fig_size = (4,2)
# title_fontsize=14
title_fontsize=13
# label_fontsize=12
label_fontsize=13
title="Number of Cases"


show_train_val_test(training_dir, validation_dir, testing_dir, title, xlabel, ylabel, figsize=fig_size, title_fontsize = title_fontsize, label_fontsize=label_fontsize)


### Preprocessing (Image Preprocessing)
#### Configuring Image Transformation Parameters for Training, Validation, Testing and  Dataset 

In [ ]:
# Get number of label/ class / category
num_class = len(os.listdir(training_dir))
print(num_class)


#Image Augmentation/ Preprocessing before training
norm=255.0
rescale=1./norm
shear_range=0.2
# shear_range=0.1
zoom_range=0.2
# zoom_range=0.1
horizontal_flip=True


# flow from directory function
# Target Image dimention
# target_size=(224, 224) # used previously
target_size=(299, 299) # default expected image dimension for inceptionv3

# Batch size
# train
batch_size=32
# batch_size=64
# batch_size=128

# validation
validation_batch_size=batch_size
# validation_batch_size=1
# test
test_batch_size=batch_size
# test_batch_size=1

# shuffle
# test
test_shuffle=False

# validation split for train
validation_split=0.0

# class mode
# class_mode='binary'
class_mode='categorical'
# class_mode='sparse'


classes = ['Normal', 'Cancer']

### Image Transformation for Training, Validation, Testing and  Dataset 

In [ ]:
train_generator = get_transformed_image_batch(training_dir, target_size, classes, class_mode=class_mode, batch_size=batch_size, rescale=rescale, shear_range=shear_range, zoom_range=zoom_range, horizontal_flip=horizontal_flip)       

validation_generator = get_transformed_image_batch(validation_dir, target_size, classes, class_mode=class_mode, batch_size=validation_batch_size, rescale=rescale)       

test_generator = get_transformed_image_batch(validation_dir, target_size, classes, class_mode=class_mode, batch_size=test_batch_size, shuffle = test_shuffle, rescale=rescale) 

### Weight Adjustmanet for Class Label Imbalance

In [ ]:
y=train_generator.classes
class_weight=get_class_weight(y)
# class_weight=None


##  Training Configuration

In [ ]:
# setting model and log output directory
# model_dir=output_directory + r"models/"+time.strftime('%Y%m%d%H%M%S')+"/"
model_dir=output_directory + r"models/"+time.strftime('%Y-%m-%d %H--%M--%S')+"/"
# log_dir=output_directory + r"logs/"+time.strftime('%Y%m%d%H%M%S')
log_dir=output_directory + r"logs/"+time.strftime('%Y-%m-%d %H--%M--%S')

create_directory(model_dir, remove=True)
create_directory(log_dir, remove=True)


init_model_file=model_dir+"base-"+"{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
model_file=model_dir+"{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
retrain_model_file=model_dir+"retrain-{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"


## Custom Model Configuration

In [ ]:
input_shape=(3,150,150)
activation='relu'
activation2='sigmoid'
padding="same"
padding2="valid"
pool_size=(2, 2)
dilation_rate=(2, 2)

### Base Model Configuration

In [ ]:
##### Base Model - InceptionV3 (pretrained) initial training settings

# inception base top layer discarded
include_top=False

# number of layers freezed
non_trainable_index = 249

init_optimizer=optimizers.Adam()
# init_optimizer=optimizers.Adam(0.000001)

print_layers=False

# initial epochs on only output layers
# init_epochs=1
init_epochs=15

# verbose
init_verbose=0

# callbacks
init_callbacks=None

# model report
print_layers=True

### Model Parameters

In [ ]:
### Full model training parameter configuration for Loss, Optimizer and Performance Metrics

# optimizer
# adam lr=0.01/0.001/0.0001/0.00001/0.000001, decay = decay=1e-5/ 1e-6
optimizer=optimizers.Adam()
# optimizer=optimizers.Adam(0.1)
# optimizer=optimizers.Adam(0.01)
# optimizer=optimizers.Adam(0.001)
# optimizer=optimizers.Adam(0.00001)
optimizer=optimizers.Adam(0.00001, decay=1e-6)



# loss function
# loss='binary_crossentropy'
loss='categorical_crossentropy'


# performance metrics ('accuracy', 'binary_accuracy', precision, recall)
metrics=['accuracy']
# metrics=['mae', 'acc']
# metrics=['mse', 'acc']
# metrics=None


### Trainning Parameters

In [ ]:
### Main model training parameter configuration

# epochs = 20/30/50
# epochs = 30 #######original
epochs = 10

# steps
steps_per_epoch=len(train_generator)
validation_steps=len(validation_generator)

# verbose 0=nothing 1=each line
# verbose=0
verbose=1


### Callbacks Configuration

In [ ]:
#### Configuration for Callbacks - CheckPoint, ReduceLROnPlateau, Early Stopping, TensorBoard

# base_logger
base_logger_stateful_metrics=None

# TerminateOnNaN

# ProgbarLogger
progbar_logger_count_mode='samples'
progbar_logger_stateful_metrics=None

# History

# LearningRateScheduler
lr_schedule = None
lr_scheduler_verbose=0

# CSVLogger
CSV_logger_filename = log_dir+ "\\csv_logger.csv"
CSV_logger_separator=','
CSV_logger_append=False


# checkpoint
# ck_monitor='val_acc'
ck_monitor='val_loss'
ck_verbose=0
ck_save_best_only=False
ck_save_weights_only=False
ck_mode='auto'
ck_period=1


# ReduceLROnPlateau
red_lr_monitor='val_loss'
# red_lr_factor=0.1 # default
red_lr_factor=0.5 # default
# red_lr_patience=10
# red_lr_patience=5
red_lr_patience=2
red_lr_verbose=1
red_lr_mode='auto'
red_lr_min_delta=0.0001
red_lr_cooldown=0
red_lr_min_lr=0.0


# early_stopping
es_monitor = 'val_loss'
es_monitor = 'val_acc'
es_min_delta=0
# es_patience=0
es_patience=5
es_verbose=0
es_mode='auto'
es_baseline=None


# tensorboard
tb_histogram_freq=0
tb_batch_size=batch_size
tb_write_graph=True
tb_write_grads=False
tb_write_images=False
tb_embeddings_freq=0
tb_embeddings_layer_names=None
tb_embeddings_metadata=None
tb_embeddings_data=None
update_freq='epoch'

### Callbacks

In [ ]:
base_logger = keras.callbacks.BaseLogger(stateful_metrics=base_logger_stateful_metrics)
terminate_on_NaN = keras.callbacks.TerminateOnNaN()
progbar_logger = keras.callbacks.ProgbarLogger(count_mode=progbar_logger_count_mode, stateful_metrics=progbar_logger_stateful_metrics)
history = keras.callbacks.History()
# learning_rate_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, lr_scheduler_verbose=0)
CSV_logger = keras.callbacks.CSVLogger(CSV_logger_filename, separator=CSV_logger_separator, append=CSV_logger_append)

### Callbacks - CheckPoint, ReduceLROnPlateau, Early Stopping, TensorBoard

In [ ]:
checkpoint = ModelCheckpoint(model_file, monitor=ck_monitor, verbose=ck_verbose, save_best_only=ck_save_best_only, save_weights_only=ck_save_weights_only, mode=ck_mode, period=ck_period)

reduce_lr = ReduceLROnPlateau(monitor=red_lr_monitor, factor=red_lr_factor, patience=red_lr_patience, verbose=red_lr_verbose, mode=red_lr_mode, min_delta=red_lr_min_delta, cooldown=red_lr_cooldown, min_lr=red_lr_min_lr)

early_stopping = EarlyStopping(monitor=es_monitor, min_delta=es_min_delta, patience=es_patience, verbose=es_verbose, mode=es_mode, baseline=es_baseline)

tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=tb_histogram_freq, batch_size=tb_batch_size, write_graph=tb_write_graph, write_grads=tb_write_grads, write_images=tb_write_images, embeddings_freq=tb_embeddings_freq, embeddings_layer_names=tb_embeddings_layer_names, embeddings_metadata=tb_embeddings_metadata, embeddings_data=tb_embeddings_data)

### Compiling Callbacks
#### ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard, 
#### BaseLogger, TerminateOnNaN , ProgbarLogger,  History, LearningRateScheduler, CSVLogger, 

In [ ]:
# base_logger, terminate_on_NaN , progbar_logger,  history, learning_rate_scheduler, CSV_logger, checkpoint, reduce_lr, early_stopping, tensorboard 

#### Base Model Callbacks

In [ ]:
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN , progbar_logger,  history, learning_rate_scheduler, CSV_logger]init_callbacks = None
init_callbacks = None
# init_callbacks = [checkpoint, tensorboard]
# init_callbacks = [checkpoint, reduce_lr, tensorboard]
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, history]
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history]
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history, CSV_logger]
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, progbar_logger, history, CSV_logger]
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, CSV_logger]
# init_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, learning_rate_scheduler, CSV_logger]

#### Main Model Callbacks

In [ ]:
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, history, CSV_logger]
callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]


# callbacks = None
# callbacks = [checkpoint, tensorboard]
###callbacks = [checkpoint, reduce_lr, tensorboard]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, history]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history, CSV_logger]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, progbar_logger, history, CSV_logger]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, CSV_logger]
# callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, learning_rate_scheduler, CSV_logger]

####  Retrain Main Model Callbacks

In [ ]:
# retrain_callbacks = None
# retrain_callbacks = [checkpoint, tensorboard]
# retrain_callbacks = [checkpoint, reduce_lr, tensorboard]
retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, history]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history, CSV_logger]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, progbar_logger, history, CSV_logger]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, CSV_logger]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, learning_rate_scheduler, CSV_logger]

## Training Base Model

In [ ]:
# get inception model
date_time(1)
init_epochs = 100
print_layers=False
model = get_inception_model(train_generator, validation_generator, init_epochs, init_verbose, init_optimizer, loss, metrics, tensorboard, init_callbacks, num_class, include_top, non_trainable_index, print_layers)
main_model = model
date_time(1)

### Base model Performance with Minimum Pre-Training

In [ ]:
# print_report=True

# y_preds, y_classes, CM, CM_report, cls_report_print = predict_report(model, test_generator, classes, print_report)

# accuracy, loss =  model_evaluate(model, test_generator, print_report)
# print(accuracy, loss)
# res=show_confusion_matrix(test_generator, y_classes, classes)

In [ ]:
# model = build_model()

## Training Base Model for Fine-Tuning

In [ ]:
# train inception model
# fine-tuning the top layers
# compile model with loss, optimizer and metrics 

# epochs=100
# optimizer=optimizers.adam(lr=0.0001)
# model.compile(optimizer, loss=loss, metrics=metrics)
# tensorboard.set_model(model) 


###################main################
# date_time(1)
# history = model.fit_generator(
#     train_generator,
#     steps_per_epoch = steps_per_epoch,
#     epochs=epochs,
#     callbacks=callbacks,
#     validation_data=validation_generator,
#     validation_steps=validation_steps,
#     class_weight=class_weight)

# date_time(1)
 
    
###################test####################
date_time(1)
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 60,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=20,
    class_weight=class_weight)

date_time(1)


### Model Performance Visualization over the Epochs

In [ ]:
plot_val=[['acc', 'val_acc'], ['loss', 'val_loss']]

title = ['Model accuracy', 'Model loss']

xlabel = ['Epoch', 'Epoch']
ylabel = ['Accuracy', 'Loss']

legend = ['Train', 'Val']

fig_size=(15, 5)


title_fontsize=17
label_fontsize=15

plot_history(history, plot_val, title, xlabel, ylabel, fig_size=fig_size, title_fontsize=title_fontsize, label_fontsize=label_fontsize)

In [ ]:
result  = model.evaluate_generator(test_generator, steps=len(test_generator), verbose=1)

# y_preds, y_classes, CM, CM_report, cls_report_print = predict_report(model, test_generator, classes, print_report)

# accuracy, loss =  model_evaluate(model, test_generator, print_report)
# print(accuracy, loss)
# res=show_confusion_matrix(test_generator, y_classes, classes)

In [ ]:
print(result[0])
print(result[1]*100, "%")

In [ ]:
y_pred = model.predict_generator(test_generator, steps=len(test_generator), verbose=1)

In [ ]:
y_classes=y_pred.argmax(axis=-1)

In [ ]:
CM = confusion_matrix(test_generator.classes, y_classes)
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(10,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(len(classes)), classes, fontsize=12)
plt.yticks(range(len(classes)), classes, fontsize=12)
plt.show()

In [ ]:
# model = build_model()

## Test Performance of All Models on Test Dataset

In [ ]:
date_time(1)

details = True
class_name = "Cancer"

report_type = "full"
results1, results2, report = test_all_models(model_dir, details, report_type, classes, class_name=class_name)

date_time(1)

### Visualization of  Performance Over All Epochs/Models based on Accuracy and Loss

In [ ]:
model_filename_list=[]
model_file_path_list=[]
for model_filename in results2:
    model_filename_list.append(model_filename)
    model_file_path_list.append(model_dir+model_filename)

In [ ]:
metric_array_names = ['Normal-precision', 'Normal-recall', 'Normal-f1-score', 
                     'Cancer-precision','Cancer-recall', 'Cancer-f1-score', 
                     'micro avg-precision', 'micro avg-recall', 'micro avg-f1-score', 
                     'macro avg-precision', 'macro avg-recall', 'macro avg-f1-score', 
                     'weighted avg-precision', 'weighted avg-recall', 'weighted avg-f1-score',
                     'Accuracy', 'Loss']
metric_array_list=[]


# 0
negative_precision_list = [results2[i][1]['Normal']['precision'] for i in results2]
metric_array_list.append(negative_precision_list)

# 1
negative_recall_list = [results2[i][1]['Normal']['recall'] for i in results2]
metric_array_list.append(negative_recall_list)

# 2
negative_f1_score_list = [results2[i][1]['Normal']['f1-score'] for i in results2]
metric_array_list.append(negative_f1_score_list)


# 3
positive_precision_list = [results2[i][1]['Cancer']['precision'] for i in results2]
metric_array_list.append(positive_precision_list)

# 4
positive_recall_list = [results2[i][1]['Cancer']['recall'] for i in results2]
metric_array_list.append(positive_recall_list)

# 5
positive_f1_score_list = [results2[i][1]['Cancer']['f1-score'] for i in results2]
metric_array_list.append(positive_f1_score_list)



# 6
micro_precision_list = [results2[i][1]['micro avg']['precision'] for i in results2]
metric_array_list.append(micro_precision_list)

# 7
micro_recall_list = [results2[i][1]['micro avg']['recall'] for i in results2]
metric_array_list.append(micro_recall_list)

# 8
micro_f1_score_list = [results2[i][1]['micro avg']['f1-score'] for i in results2]
metric_array_list.append(micro_f1_score_list)



# 9
macro_precision_list = [results2[i][1]['macro avg']['precision'] for i in results2]
metric_array_list.append(macro_precision_list)

# 10
macro_recall_list = [results2[i][1]['macro avg']['recall'] for i in results2]
metric_array_list.append(macro_recall_list)

# 11
macro_f1_score_list = [results2[i][1]['macro avg']['f1-score'] for i in results2]
metric_array_list.append(macro_f1_score_list)



# 12
weighted_precision_list = [results2[i][1]['weighted avg']['precision'] for i in results2]
metric_array_list.append(negative_f1_score_list)

# 13
weighted_recall_list = [results2[i][1]['weighted avg']['recall'] for i in results2]
metric_array_list.append(negative_f1_score_list)

# 14
weighted_f1_score_list = [results2[i][1]['weighted avg']['f1-score'] for i in results2]
metric_array_list.append(negative_f1_score_list)



# 15
accuracy_list = [results1[i][0] for i in results1]
metric_array_list.append(accuracy_list)

# 16
loss_list = [results1[i][1]for i in results1]
metric_array_list.append(loss_list)


In [ ]:
metric_array_list_percent = metric_array_list
num_metrics = len(metric_array_list)
for i in range(num_metrics):
    if i!=16:
        metric_array_list_percent[i] = [i*100 for i in metric_array_list[i]]

In [ ]:
# metric_array_list_norm = metric_array_list
# for i in range(len(metric_array_list)):
#     m=max(metric_array_list[i])
#     metric_array_list_norm[i] = [i/m for i in metric_array_list_norm[i]]

In [ ]:
skip_index = [1,2,7,8,10,11]
plot_index = [2, 3,4,5, 12, 13, 14, 15]

# plot_index = [2, 3,4,5, 12, 13, 14, 15]

In [ ]:
figsize_col = 12
figsize_row = 4

facecolor='w'
edgecolor='k'

titlesize = 'Large'

In [ ]:
seperate_plot = False
# seperate_plot = True

filter_skip = False

filter_plot = False
filter_plot = True

dpi=150


length=len(metric_array_list)
num_epochs=len(results2)
x = np.arange(num_epochs)

fig, axs = plt.subplots(figsize=(18, 12), dpi=dpi, facecolor=facecolor, edgecolor=edgecolor)
plt.rcParams.update({'axes.titlesize': titlesize})
for i in range(length):
    if not filter_plot or i in plot_index:
        if seperate_plot:  
            fig, axs = plt.subplots(figsize=(figsize_col, figsize_row), dpi=dpi, facecolor=facecolor, edgecolor=edgecolor)
            plt.rcParams.update({'axes.titlesize': titlesize})
            
            plt.plot(x, metric_array_list_percent[i], label=metric_array_names[i])
            plt.title(metric_array_names[i])
            plt.xlabel("Epoch")
            plt.ylabel("Score (100%)")
#             plt.yticks(np.arange(0, 100, 5))
#             plt.xticks(np.arange(0, num_epochs, 1))
            plt.legend()
            plt.grid(True)

#             plt.ylim([min(metric_array_list_percent[i]),max(metric_array_list_percent[i])])
            plt.show()
        else:
            plt.plot(x, metric_array_list_percent[i], label=metric_array_names[i])
            plt.title(metric_array_names[i])
        

            
if not seperate_plot:
    plt.xlabel("Epoch")
    plt.ylabel("Val")
#     plt.yticks(np.arange(0, 100, 5))
#     plt.xticks(np.arange(0, num_epochs, 1))
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
figsize_col=10
figsize_row=3
fig, axs = plt.subplots(figsize=(figsize_col, figsize_row), dpi=dpi, facecolor=facecolor, edgecolor=edgecolor)
plt.rcParams.update({'axes.titlesize': titlesize})


x = np.arange(len(results1))


plt.subplot(121)
plt.plot(x, accuracy_list, label="Accuarcy")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Score(100%)")
plt.legend()

plt.subplot(122)
plt.plot(x, loss_list, label= "Loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
nums = [3,13,17, 18]
num_model = 0

In [ ]:
for i in range(len(metric_array_list_percent)):
    print("%6s%10s%.2f"%(metric_array_names[i],":", metric_array_list_percent[i][num_model]))

In [ ]:
# model_dir="data\\output\\models\\20181201035451\\"
# model=keras.models.load_model(model_dir+"30-val_acc-0.85-val_loss-0.66.hdf5")

model_path = model_file_path_list[num_model]

model = keras.models.load_model(model_path)

accuracy, loss =  model_evaluate(model, test_generator, print_report=True)
y_preds, y_classes, CM, cls_report, cls_report_print = predict_report(model, test_generator, classes, print_report=True)

In [ ]:
classes = ['Normal', 'PNEUMONIA']
CM = confusion_matrix(test_generator.classes, y_classes)
fig, ax = plot_confusion_matrix(conf_mat=CM , figsize=(10,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(len(classes)), classes, fontsize=12)
plt.yticks(range(len(classes)), classes, fontsize=12)
plt.show()

# Retraining Best Model

In [ ]:
# reset_callbacks(checkpoint, reduce_lr, early_stopping, tensorboard)
# reset_graph(model)

## Configuration for Retraining

### Model Retraining Parameters

In [ ]:
### Full model training parameter configuration for Loss, Optimizer and Performance Metrics

# optimizer
# adam lr=0.01/0.001/0.0001/0.00001/0.000001, decay = decay=1e-5/ 1e-6
optimizer=optimizers.Adam()
# optimizer=optimizers.Adam(0.1)
# optimizer=optimizers.Adam(0.01)
# optimizer=optimizers.Adam(0.001)
# optimizer=optimizers.Adam(0.00001)
# optimizer=optimizers.Adam(0.00001, decay=1e-7)



# loss function
# loss='binary_crossentropy'
loss='categorical_crossentropy'


# performance metrics ('accuracy', 'binary_accuracy', precision, recall)
metrics=['accuracy']
# metrics=['mae', 'acc']
# metrics=['mse', 'acc']
# metrics=None


### Retrainning Parameters

In [ ]:
### Main model training parameter configuration

# epochs = 20/30/50
epochs = 30

# steps
steps_per_epoch=len(train_generator)
validation_steps=len(validation_generator)

# verbose 0=nothing 1=each line
verbose=0



### Callbacks Configuration

In [ ]:
#### Configuration for Callbacks - CheckPoint, ReduceLROnPlateau, Early Stopping, TensorBoard

# base_logger
base_logger_stateful_metrics=None

# TerminateOnNaN

# ProgbarLogger
progbar_logger_count_mode='samples'
progbar_logger_stateful_metrics=None

# History

# LearningRateScheduler
lr_schedule = None
lr_scheduler_verbose=0

# CSVLogger
CSV_logger_filename = log_dir+ "\\csv_logger.csv"
CSV_logger_separator=','
CSV_logger_append=False


# checkpoint
# ck_monitor='val_acc'
ck_monitor='val_loss'
ck_verbose=0
ck_save_best_only=False
ck_save_weights_only=False
ck_mode='auto'
ck_period=1


# ReduceLROnPlateau
red_lr_monitor='val_loss'
red_lr_factor=0.1 # default
red_lr_patience=10
red_lr_verbose=1
red_lr_mode='auto'
red_lr_min_delta=0.0001
red_lr_cooldown=0
red_lr_min_lr=0.0


# early_stopping
es_monitor = 'val_loss'
es_min_delta=0
# es_patience=0
es_patience=5
es_verbose=0
es_mode='auto'
es_baseline=None


# tensorboard
tb_histogram_freq=0
tb_batch_size=batch_size
tb_write_graph=True
tb_write_grads=False
tb_write_images=False
tb_embeddings_freq=0
tb_embeddings_layer_names=None
tb_embeddings_metadata=None
tb_embeddings_data=None
update_freq='epoch'

### Callbacks

In [ ]:
base_logger = keras.callbacks.BaseLogger(stateful_metrics=base_logger_stateful_metrics)
terminate_on_NaN = keras.callbacks.TerminateOnNaN()
progbar_logger = keras.callbacks.ProgbarLogger(count_mode=progbar_logger_count_mode, stateful_metrics=progbar_logger_stateful_metrics)
history = keras.callbacks.History()
# learning_rate_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, lr_scheduler_verbose=0)
CSV_logger = keras.callbacks.CSVLogger(CSV_logger_filename, separator=CSV_logger_separator, append=CSV_logger_append)

### Callbacks - CheckPoint, ReduceLROnPlateau, Early Stopping, TensorBoard

In [ ]:
checkpoint = ModelCheckpoint(model_file, monitor=ck_monitor, verbose=ck_verbose, save_best_only=ck_save_best_only, save_weights_only=ck_save_weights_only, mode=ck_mode, period=ck_period)

reduce_lr = ReduceLROnPlateau(monitor=red_lr_monitor, factor=red_lr_factor, patience=red_lr_patience, verbose=red_lr_verbose, mode=red_lr_mode, min_delta=red_lr_min_delta, cooldown=red_lr_cooldown, min_lr=red_lr_min_lr)

early_stopping = EarlyStopping(monitor=es_monitor, min_delta=es_min_delta, patience=es_patience, verbose=es_verbose, mode=es_mode, baseline=es_baseline)

tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=tb_histogram_freq, batch_size=tb_batch_size, write_graph=tb_write_graph, write_grads=tb_write_grads, write_images=tb_write_images, embeddings_freq=tb_embeddings_freq, embeddings_layer_names=tb_embeddings_layer_names, embeddings_metadata=tb_embeddings_metadata, embeddings_data=tb_embeddings_data)

### Compiling Callbacks
#### ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard, 
#### BaseLogger, TerminateOnNaN , ProgbarLogger,  History, LearningRateScheduler, CSVLogger, 

In [ ]:
# base_logger, terminate_on_NaN , progbar_logger,  history, learning_rate_scheduler, CSV_logger, checkpoint, reduce_lr, early_stopping, tensorboard 

####  Retrain Main Model Callbacks

In [ ]:
# retrain_callbacks = None
# retrain_callbacks = [checkpoint, tensorboard]
# retrain_callbacks = [checkpoint, reduce_lr, tensorboard]
retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, history]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, history, CSV_logger]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, terminate_on_NaN, progbar_logger, history, CSV_logger]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, CSV_logger]
# retrain_callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard, base_logger, terminate_on_NaN, progbar_logger, history, learning_rate_scheduler, CSV_logger]

## Retraining Best Model
### Selecting best model file based on validation accuracy mentioned in file names

In [ ]:
# selecting best model file / checkpoint for retraining
# model_path = model_dir+r"12-val_acc-0.70-val_loss-1.09.hdf5"
# model_path = model_dir+r"20-val_acc-0.66-val_loss-1.97.hdf5"

# best accuracy/ F-1 score
# model_path = "data/output/models/"+"17-val_acc-0.82-val_loss-0.42.hdf5"

# Lowest validation Loss
# model_path = "data/output/models/"+"12-val_acc-0.70-val_loss-1.09.hdf5"

# Best Recall
# model_path = "data/output/models/"+"20-val_acc-0.66-val_loss-1.97.hdf5"

model_path = model_dir+r"20-val_acc-0.71-val_loss-1.26.hdf5"

model = keras.models.load_model(model_path)

# train inception model
# fine-tuning the top layers
# compile model with loss, optimizer and metrics 
model.compile(optimizer, loss=loss, metrics=metrics)
tensorboard.set_model(model) 

# retrain by loading last good model
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_per_epoch,
    epochs=epochs,
    # verbose=1,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    class_weight=class_weight,
    initial_epoch=initial_epoch)
 

In [ ]:
def extract_id(x):
    
    # split into a list
    a = x.split('/')
    # split into a list
    b = a[1].split('.')
    extracted_id = b[0]
    
    return extracted_id



test_filenames = test_generator.filenames
df_preds = pd.DataFrame(predictions, columns=classes)
df_preds['file_names'] = test_filenames
df_preds['id'] = df_preds['file_names'].apply(extract_id)
df_preds.head()

# Get the true labels
y_true = test_generator.classes

# Get the predicted labels as probabilities
y_pred = df_preds['Cancer']

fpr_keras, tpr_keras, thresholds_keras = roc_curve(test_gen.classes, y_pred_keras)
auc_keras = auc(fpr_keras, tpr_keras)

roc_auc_score(y_true, y_pred)


In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='area = {:.3f}'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
submission = pd.DataFrame({'id':image_id, 
                           'label':y_pred, 
                          }).set_index('id')

submission.to_csv('patch_preds.csv', columns=['label']) 
submission.head()

##### Retriving actual labels 

In [ ]:
label_map = (test_generator.class_indices)
label_map_rev = {v: name_correct(k) for k,v in label_map.items()}
num_batch_t = len(test_generator)
print(label_map)
print(label_map_rev)

##### Showing accuracy for Model over Single Batch of Test Dataset

In [ ]:
num = random.randint(0, num_batch_t-1)
y_img_batch, y_class_batch = test_generator[num] 
y_pred = np.argmax(model.predict(y_img_batch),-1)
y_true = np.argmax(y_class_batch,-1)
print("Selected Batch No: %d\nBatch Size: %d"%(num, len(y_pred)))
print("Accuracy : ", sum(y_pred==y_true)/batch_size*100, "%")

y_true_labels = [label_map_rev[c] for c in y_true]
y_pred_labels = [label_map_rev[c] for c in y_pred]
batch_size_t = len(y_true_labels)

### Visualization 
Visualization of performance of a random test dataset batch and few random images from a batch

#### Visualization 1 (Random Batch)
Visualization of performance of a random test dataset batch

Setting parameters for visualization 1

In [ ]:
figure_directory = "data/output/figures"
image_file_name = figure_directory+"/result"

dpi=100

update_image = True


cols = 8
rows= batch_size_t/cols
if batch_size_t%cols==0:
    rows = int(batch_size_t/cols)
else:
    rows = int(batch_size_t/cols)+1
    
figsize_col = cols*2.5
figsize_row = rows*2.5

hspace = 0.5
wspace = 0.3

facecolor='w'
edgecolor='k'

titlesize = 'small'

true_prediction_label_color='black'
false_prediction_label_color='red'

true_label_title_prefix = "org : "
pred_label_title_prefix = "pred: "

if not os.path.exists(figure_directory):
    os.mkdir(figure_directory)

#### Visualization 1 (Random Batch)
Visualization of performance of a random test dataset batch

In [ ]:
# figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')
fig, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(figsize_col, figsize_row),
                        dpi=dpi, facecolor=facecolor, edgecolor=edgecolor,
                        subplot_kw={'xticks': [], 'yticks': []})


plt.rcParams.update({'axes.titlesize': titlesize})
plt.subplots_adjust(hspace=hspace, wspace=wspace)

for i in range(0, batch_size_t): # how many imgs will show from the mxn grid
    plt.subplot(rows, cols, i+1)
    
    plt.imshow(y_img_batch[i])
    
    plt.xticks([])
    plt.yticks([])
    
    if y_true_labels[i]==y_pred_labels[i]:
        plt.title(true_label_title_prefix + y_true_labels[i] + "\n" + pred_label_title_prefix + y_pred_labels[i])
    else:
        plt.title(true_label_title_prefix + y_true_labels[i] + "\n" + pred_label_title_prefix + y_pred_labels[i], color=false_prediction_label_color)
        
    if update_image and os.path.exists(image_file_name):
        os.remove(image_file_name)
    
    fig.savefig(image_file_name, dpi=dpi)

plt.tight_layout()
plt.show()

#### Visualization 2 (Random) 
Visualization of performance of a few random images from a random batch

Setting parameters for visualization 2

In [ ]:
figure_directory = "data/output/figures"
image_file_name = figure_directory+"/sample"

dpi=100

update_image = True

cols = 4
rows= 2

count = rows*cols
    
figsize_col = cols*2.5
figsize_row = rows*2.5

hspace = 0.5
wspace = 0.3

# titlesize = 'small'

true_prediction_label_color='black'
false_prediction_label_color='red'

true_label_title_prefix = "org:  "
pred_label_title_prefix = "pred: "

#### Visualization 2 (Random) 
Visualization of performance of a few random images from a random batch

In [ ]:
# figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')
fig, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(figsize_col, figsize_row),
                        dpi=dpi, facecolor=facecolor, edgecolor=edgecolor,
                        subplot_kw={'xticks': [], 'yticks': []})

plt.rcParams.update({'axes.titlesize': titlesize})
plt.subplots_adjust(hspace=hspace, wspace=wspace)


batch_size_tmp = batch_size_t

m = {}

for i in range(0, count): 
    num = random.randint(0, batch_size_tmp-1)
    while num in m:
        num = random.randint(0, batch_size_tmp-1)
    
    m[num]=1
    
    plt.subplot(rows, cols, i+1)
    
    plt.imshow(y_img_batch[num])
    
    plt.xticks([])
    plt.yticks([])
    
    if y_true_labels[num]==y_pred_labels[num]:
        plt.title(true_label_title_prefix + y_true_labels[num] + "\n" + pred_label_title_prefix + y_pred_labels[num])
    else:
        plt.title(true_label_title_prefix + y_true_labels[num] + "\n" + pred_label_title_prefix + y_pred_labels[num], color=false_prediction_label_color)
    
   
    if update_image and os.path.exists(image_file_name):
        os.remove(image_file_name)   
    
    fig.savefig(image_file_name, dpi=dpi)


plt.tight_layout()
plt.show()